In [1]:
import sys
sys.path.append('../')

In [4]:
import os
from glob import glob
from src.pipeline.ml.deeplabv3 import DeepLabV3Plus
import tensorflow as tf

ModuleNotFoundError: No module named 'src.pipeline.ml.deeplabv3.deeplabv3'; 'src.pipeline.ml.deeplabv3' is not a package

In [3]:
model = DeepLabV3Plus(image_size=512, num_classes=9, channels=4)
model.summary()

NameError: name 'DeepLabV3Plus' is not defined

In [4]:
NUM_TRAIN_IMAGES = 56944
NUM_VAL_IMAGES = 18334
TRAIN_DATA_DIR = "../supervised/Agriculture-Vision-2021/train/"
VAL_DATA_DIR = "../supervised/Agriculture-Vision-2021/val/"

train_images = sorted(glob(os.path.join(TRAIN_DATA_DIR, "images/rgb/*")))[:NUM_TRAIN_IMAGES]
train_masks = sorted(glob(os.path.join(TRAIN_DATA_DIR, "masks/*")))[:NUM_TRAIN_IMAGES]

val_images = sorted(glob(os.path.join(VAL_DATA_DIR, "images/rgb/*")))[:NUM_VAL_IMAGES]
val_masks = sorted(glob(os.path.join(VAL_DATA_DIR, "masks/*")))[:NUM_VAL_IMAGES]


In [9]:
train_labels = sorted(glob(os.path.join(TRAIN_DATA_DIR, "labels/*/*")))

In [10]:
len(train_labels)

512496

In [11]:
512496 / 9

56944.0

In [14]:
IMAGE_SIZE = 512
BATCH_SIZE = 32
def read_image(image_path, mask=False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tf.image.decode_png(image, channels=1)
        image.set_shape([None, None, 1])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    else:
        image = tf.image.decode_png(image, channels=3)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = tf.keras.applications.resnet50.preprocess_input(image)
    return image

def load_data(image_list, mask_list, labels_list):
    image = read_image(image_list)
    mask = read_image(mask_list, mask=True)
    return image, mask


def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

In [15]:
train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)
print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)

Train Dataset: <BatchDataset element_spec=(TensorSpec(shape=(32, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 512, 512, 1), dtype=tf.float32, name=None))>
Val Dataset: <BatchDataset element_spec=(TensorSpec(shape=(32, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 512, 512, 1), dtype=tf.float32, name=None))>


In [4]:
from tensorflow import keras
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=loss,
    metrics=["accuracy"]
)
model.save('deepv3_seg')

INFO:tensorflow:Assets written to: deepv3_seg/assets


INFO:tensorflow:Assets written to: deepv3_seg/assets
